In [0]:
NUM_ROUNDS = {
    # (block_size, key_size): num_rounds
    (32, 64): 32,
    (48, 96): 36,
    (64, 128): 44,
}

In [0]:
def get_sequence(num_rounds):
    if num_rounds < 40:
        states = [1] * 5
#         print("States: ",states)
    else:
        states = [1] * 6

    for i in range(num_rounds - 5):
        if num_rounds < 40:
#             print("States of- ",i)
            feedback = states[i + 2] ^ states[i]
#             print("fb is: ",feedback)
        else:
            feedback = states[i + 1] ^ states[i]
#             priint("oh!")
        states.append(feedback)

    return tuple(states)

In [0]:
#32, 64, 0x1918111009080100
class Simeck:
    def __init__(self, block_size, key_size, master_key):
        assert (block_size, key_size) in NUM_ROUNDS
        assert 0 <= master_key < (1 << key_size)###-------------------------------------------------------
        self._block_size = block_size
        self._key_size = key_size
        self._word_size = block_size // 2
        self._num_rounds = NUM_ROUNDS[(block_size, key_size)]
#         print("Rounds are: " ,self._num_rounds)
        self._sequence = get_sequence(self._num_rounds)
#         print("seq",self._sequence)
        self._modulus = 1 << self._word_size #modulus= 2^16= 65536
#         print(self._modulus)
        self.change_key(master_key)

    def _LROT(self, x, r):
        assert 0 <= x < self._modulus
        res = (x << r) % self._modulus
        res |= x >> (self._word_size - r)
        return res

    def _round(self, round_key, left, right):
        assert 0 <= round_key < self._modulus
        assert 0 <= left < self._modulus
        assert 0 <= right < self._modulus
        temp = left
        left = right ^ (left & self._LROT(left, 5)) ^ self._LROT(left, 1) ^ round_key
        right = temp
#         print (hex(round_key))
#         print(hex(left), "----", hex(right))
        return left, right

    def change_key(self, master_key):
        print("key size: ",self._key_size)
        assert 0 <= master_key < (1 << self._key_size)
        states = []
        for i in range(self._key_size // self._word_size):
            states.append(master_key % self._modulus) #-----------------------------------[a,b,c,d?]
#             print("states2:",states)
            master_key >>= self._word_size
#             print("mk:",master_key)

        constant = self._modulus - 4
        round_keys = []
        for i in range(self._num_rounds):
            round_keys.append(states[0]) ##########
#             print("rk",round_keys)
            left, right = states[1], states[0]##############
#             print("left1: ",left)
#             print("right1: ",right)
            left, right = self._round(constant ^ self._sequence[i],############
                                      left, right)
#             print("left2: ",left)
#             print("right2: ",right)
#             print("s before: ",states)
#             print("left o/p: ")
            states.append(left) #############
#             print(states) 
            states.pop(0)  #############
            states[0] = right           #############                                          
        self.__round_keys = tuple(round_keys)

    def encrypt(self, plaintext):
        assert 0 <= plaintext < (1 << self._block_size)
        left = plaintext >> self._word_size
        right = plaintext % self._modulus

        for idx in range(self._num_rounds):
            left, right = self._round(self.__round_keys[idx],
                                      left, right)

        ciphertext = (left << self._word_size) | right
        return ciphertext


In [0]:
def print_test_vector(block_size, key_size, key, plain, cipher):
    print ('Simeck', block_size, key_size)
    print( 'key===', hex(key)[2:])
    print( 'plaintext=== ', hex(plain)[2:])
    print( 'ciphertext===', hex(cipher)[2:])

In [5]:
def main():
    plaintext32 = 0x65656877
    key64 = 0x1918111009080100
    simeck32 = Simeck(32, 64, key64)
    ciphertext32 = simeck32.encrypt(plaintext32)
    print_test_vector(32, 64, key64, plaintext32, ciphertext32)
    print

#     plaintext48 = 0x72696320646e
#     key96 = 0x1a19181211100a0908020100
#     simeck48 = Simeck(48, 96, key96)
#     ciphertext48 = simeck48.encrypt(plaintext48)
#     print_test_vector(48, 96, key96, plaintext48, ciphertext48)
#     print

#     plaintext64 = 0x656b696c20646e75
#     key128 = 0x1b1a1918131211100b0a090803020100
#     simeck64 = Simeck(64, 128, key128)
#     ciphertext64 = simeck64.encrypt(plaintext64)
#     print_test_vector(64, 128, key128, plaintext64, ciphertext64)


if __name__ == '__main__':
  main()

key size:  64
Simeck 32 64
key=== 1918111009080100
plaintext===  65656877
ciphertext=== 770d2c76


In [6]:
1<<5

32

In [0]:
xs={'a':4,'b':10,'c':5,'d':1}
xr={'p':100,'q':200,'r':300}

In [0]:
z={**xs, **xr}
z

{'a': 4, 'b': 10, 'c': 5, 'd': 1, 'p': 100, 'q': 200, 'r': 300}

In [0]:
print(sorted(xs.items(),key=lambda x:x[1]))

[('d', 1), ('a', 4), ('c', 5), ('b', 10)]


In [0]:
s=['a','b',1,2,3]
left=0
right=len(s)-1
while(left<right):
  s[left],s[right]=s[right],s[left]
  left=left+1
  right-=1